In [ ]:
import pandas   as pd
import polars   as pl
import numpy    as np
import networkx as nx
import squarify
import rtsvg
rt      = rtsvg.RACETrack()
relates = [('fm','to')]

# Read in the simple graph templates
df      = pl.read_csv('../rtsvg/config/simple_graphs_df.csv')
g       = rt.createNetworkXGraph(df, relates)

# Create the component lookup structure
comp_lu = {}
for _node_set_ in nx.connected_components(g):
    _g_              = g.subgraph(_node_set_)
    _nodes_, _edges_ = _g_.number_of_nodes(), _g_.number_of_edges()
    if _nodes_ not in comp_lu:          comp_lu[_nodes_]          = {}
    if _edges_ not in comp_lu[_nodes_]: comp_lu[_nodes_][_edges_] = []
    comp_lu[_nodes_][_edges_].append(_g_)

# Positional Information
_df_    = pd.read_csv('../rtsvg/config/simple_graph_layouts.csv')
pos_templates  = {}
for i in range(len(_df_)): pos_templates[_df_['node'][i]] = [_df_['x'][i], _df_['y'][i]]

# Sanity Check
set(df['fm']) | set(df['to']) == set(pos_templates.keys())

In [ ]:
#pos = rt.treeMapGraphComponentPlacement(g, pos_templates, bounds_percent=0.3)
#rt.link(df, relates, pos, w=1200,h=900)

In [ ]:
def layoutSimpleTemplates(self, g, pos):
    for _node_set_ in nx.connected_components(g):
        _g_              = g.subgraph(_node_set_)
        _nodes_, _edges_ = _g_.number_of_nodes(), _g_.number_of_edges()
        if _nodes_ in comp_lu and _edges_ in comp_lu[_nodes_]:
            for _g_template_ in comp_lu[_nodes_][_edges_]:
                if nx.is_isomorphic(_g_, _g_template_):
                    gm     = nx.isomorphism.GraphMatcher(_g_, _g_template_)
                    _dict_ = next(gm.subgraph_isomorphisms_iter())
                    for k in _dict_.keys(): pos[k] = pos_templates[_dict_[k]]
                    break
    return self.treeMapGraphComponentPlacement(g, pos, bounds_percent=0.3)

df2   = pl.DataFrame({'src':['a','b','c','d','e','f'],'dst':['b','c','a','e','f','d']})
g_df2 = rt.createNetworkXGraph(df2, [('src','dst')])
pos   = {}
pos   = layoutSimpleTemplates(rt, g_df2, pos)
rt.link(df2, [('src','dst')], pos)